# Data preparation and customer analytics

In [1]:
# Ncessary imports
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Data loading
purchase = pd.read_csv('../data/raw/QVI_purchase_behaviour.csv')
transaction = pd.read_excel('../data/raw/QVI_transaction_data.xlsx')

# View the purchase behavior data
purchase.head(3)

,LYLTY_CARD_NBR,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,YOUNG SINGLES/COUPLES,Premium
1,1002,YOUNG SINGLES/COUPLES,Mainstream
2,1003,YOUNG FAMILIES,Budget


In [3]:
# View the transaction data
transaction.head(3)

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9


In [4]:
# Inspecting the data
print(purchase.info())
print('---')
print(transaction.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72637 entries, 0 to 72636
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   LYLTY_CARD_NBR    72637 non-null  int64 
 1   LIFESTAGE         72637 non-null  object
 2   PREMIUM_CUSTOMER  72637 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB
None
---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264836 entries, 0 to 264835
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DATE            264836 non-null  int64  
 1   STORE_NBR       264836 non-null  int64  
 2   LYLTY_CARD_NBR  264836 non-null  int64  
 3   TXN_ID          264836 non-null  int64  
 4   PROD_NBR        264836 non-null  int64  
 5   PROD_NAME       264836 non-null  object 
 6   PROD_QTY        264836 non-null  int64  
 7   TOT_SALES       264836 non-null  float64
dtypes: float64(1), int64(6), obj

In [5]:
# Convert date column to datetime
transaction['DATE'] = pd.to_datetime(transaction['DATE'])

# Extract the pack size and remove it from the product name column
transaction['PROD_NAME'] = transaction['PROD_NAME'].str.replace(r'g$', '', regex=True)
transaction['PACK_SIZE(g)'] = transaction['PROD_NAME'].str.extract(r'(\d+)').astype(int)

# Remove pack size from the product name column
transaction['PROD_NAME'] = transaction['PROD_NAME'].str.replace(r'\d+', '', regex=True)
transaction['PROD_NAME'] = transaction['PROD_NAME'].str.strip()

# Extract the brand name (assuming it is the first word)
transaction['BRAND'] = transaction['PROD_NAME'].str.split().str[0]

# Remove the brand name from the product name column
transaction['PROD_NAME'] = transaction['PROD_NAME'].apply(lambda x: ' '.join(x.split()[1:]))
# transaction['PROD_NAME'] = transaction['PROD_NAME'].str.strip()

# View the new columns 
transaction[['PROD_NAME', 'PACK_SIZE(g)', 'BRAND']].head(3)

,PROD_NAME,PACK_SIZE(g),BRAND
0,Chip Compny SeaSalt,175,Natural
1,Nacho Cheese,175,CCs
2,Crinkle Cut Chips Chicken,170,Smiths


In [ ]:
# Merge the datasets
data = pd.merge(transaction, purchase, on = 'LYLTY_CARD_NBR')

# Identify outliers in TOT_SALES and PROD_QTY
q1_sales, q3_sales = np.percentile(data['TOT_SALES'], [25, 75])
iqr_sales = q3_sales - q1_sales
lower_bound_sales = q1_sales - 1.5 * iqr_sales
upper_bound_sales = q3_sales + 1.5 * iqr_sales

# Remove outliers
data = data[(data['TOT_SALES'] >= lower_bound_sales) & (data['TOT_SALES'] <= upper_bound_sales)]

# Confirm if there are any missing values
data.isnull().sum()

# save the merged dataset as csv
folder_path = '../data/processed/'
os.makedirs(folder_path, exist_ok=True)
data.to_csv(f'{folder_path}QVI_merged_data.csv', index=False)

# view the dataset
data.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE(g),BRAND,LIFESTAGE,PREMIUM_CUSTOMER
0,1970-01-01 00:00:00.000043390,1,1000,1,5,Chip Compny SeaSalt,2,6.0,175,Natural,YOUNG SINGLES/COUPLES,Premium
1,1970-01-01 00:00:00.000043599,1,1307,348,66,Nacho Cheese,3,6.3,175,CCs,MIDAGE SINGLES/COUPLES,Budget
2,1970-01-01 00:00:00.000043605,1,1343,383,61,Crinkle Cut Chips Chicken,2,2.9,170,Smiths,MIDAGE SINGLES/COUPLES,Budget
3,1970-01-01 00:00:00.000043329,2,2373,974,69,Chip Thinly S/Cream&Onion,5,15.0,175,Smiths,MIDAGE SINGLES/COUPLES,Budget
4,1970-01-01 00:00:00.000043330,2,2426,1038,108,Tortilla ChpsHny&Jlpno Chili,3,13.8,150,Kettle,MIDAGE SINGLES/COUPLES,Budget


DATE                0
STORE_NBR           0
LYLTY_CARD_NBR      0
TXN_ID              0
PROD_NBR            0
PROD_NAME           0
PROD_QTY            0
TOT_SALES           0
PACK_SIZE(g)        0
BRAND               0
LIFESTAGE           0
PREMIUM_CUSTOMER    0
dtype: int64